In [1]:
import gensim
from gensim.models import Word2Vec
from nltk.parse.stanford import StanfordDependencyParser
import os
from itertools import combinations
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True,limit=90000)

C:\Users\nikhi\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#-Dependencies needed to run the Stanford Dependency Parser
os.environ['JAVAHOME'] = 'C://Program Files//Java//jdk1.8.0_144//bin//java.exe'
os.environ['STANFORD_PARSER'] = 'stanford-parser-full-2018-10-17/stanford-parser.jar'
os.environ['STANFORD_MODELS'] = 'stanford-parser-full-2018-10-17/stanford-parser-3.9.2-models.jar'
path_to_jar = 'stanford-parser-full-2018-10-17/stanford-parser.jar'
path_to_models_jar = 'stanford-parser-full-2018-10-17/stanford-parser-3.9.2-models.jar'
dep_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)

In [5]:
 #List of features which are termed as DIRECT OBJECTS:
words = ['alarm system','interior mirror','central locking system','automatic power window','manual power window','power window',
        'finger protection','exterior monitoring','LED alarm system','central alarm system']

#Converts a list into combinations
list1=[]
for elements in list(combinations(words, 2)):
    for i in elements:
        list1.append(i.split())

k=0
further_processing = []
while k < len(list1):
    sim = model.n_similarity(list1[k],list1[k+1])  #Check Similarity!
    if(sim>0.40): #Similarity Threshold
        print("The similarity threshold is more for this")
        print(list1[k],list1[k+1])
        print(sim)
        print(" ")
        S1 = set(list1[k])
        S2 = set(list1[k+1])
        if len(S1.intersection(S2)) == 0:
            print('There are no direct Objects')
            print(" ")
            print(list1[k],list1[k+1])
            a=' '.join(list1[k])
            b=' '.join(list1[k+1]) 
            further_processing.append([a,b])       
        else:
            print(" ")
            if(len(list1[k+1]) >= len(list1[k])):
                print(list1[k+1],'REQUIRE',list1[k])
            else:
                print(list1[k],'REQUIRE',list1[k+1])
        print(" ")
    k=k+2

    

The similarity threshold is more for this
['alarm', 'system'] ['central', 'locking', 'system']
0.5238854102818237
 
 
['central', 'locking', 'system'] REQUIRE ['alarm', 'system']
 
The similarity threshold is more for this
['alarm', 'system'] ['automatic', 'power', 'window']
0.43492856494405135
 
There are no direct Objects
 
['alarm', 'system'] ['automatic', 'power', 'window']
 
The similarity threshold is more for this
['alarm', 'system'] ['exterior', 'monitoring']
0.4207257592584278
 
There are no direct Objects
 
['alarm', 'system'] ['exterior', 'monitoring']
 
The similarity threshold is more for this
['alarm', 'system'] ['LED', 'alarm', 'system']
0.8153857191875945
 
 
['LED', 'alarm', 'system'] REQUIRE ['alarm', 'system']
 
The similarity threshold is more for this
['alarm', 'system'] ['central', 'alarm', 'system']
0.9060327661720962
 
 
['central', 'alarm', 'system'] REQUIRE ['alarm', 'system']
 
The similarity threshold is more for this
['interior', 'mirror'] ['exterior', 'mon

In [6]:
print(further_processing)

[['alarm system', 'automatic power window'], ['alarm system', 'exterior monitoring'], ['interior mirror', 'exterior monitoring'], ['central locking system', 'automatic power window'], ['central locking system', 'manual power window'], ['automatic power window', 'LED alarm system'], ['automatic power window', 'central alarm system'], ['exterior monitoring', 'LED alarm system'], ['exterior monitoring', 'central alarm system']]


In [7]:
sentences = ["If the window central locking is being opened, pressing the remote control button will close the window.",
          "If the window is just opened, nothing automatic power window happens by pressing the button.",
          "If the window is closed, press the alarm system remote control button to open the window.",
          "If the window is closed, nothing happens by pressing the key.",
          "If the window is fully open, pressing the remote alarm system control button will have no effect."]

sentences = ["Sample Sentences"]

In [8]:
list_process = []
dobj1 = []
dobj2 = []
dobj_diff = []

for i in list(further_processing):
    further_processing.remove(i)
    list_process.append(i)
    print(list_process)
    for z in list_process:
        for s in sentences:
            if(z[0]) in s:
                print(s)
                res = dep_parser.raw_parse(s)
                dep = list(res)[0]
                for triple in dep.triples():
                    if(triple[1] == 'dobj'):
                        print (triple[1],"(",triple[0][0],", ",triple[2][0],")")
                        d1=' '.join(triple[0][0])
                        d2=' '.join(triple[2][0])
                        dobj1.append([d1,d2])
                if(len(dobj1)== 0):
                    print(z[0],'EXCLUDES',z[1])
            
            if(z[1]) in s:
                print(s)
                res = dep_parser.raw_parse(s)
                dep = list(res)[0]
                for triple in dep.triples():
                    if(triple[1] == 'dobj'):
                        print (triple[1],"(",triple[0][0],", ",triple[2][0],")")
                        d3=' '.join(triple[0][0])
                        d4=' '.join(triple[2][0])
                        dobj2.append([d3,d4])
                
                if(len(dobj2)== 0):
                    print(z[0],'EXCLUDES',z[1])
                
                
            for element in dobj1:
                if element in dobj2:
                    dobj_diff.append(element)
                    print(len(dobj_diff))
                
        if len(dobj_diff) == 0:
            print(z[0],'EXCLUDES',z[1])
        else:
            print(z[0],'REQUIRE',z[1])
                
                
        list_process = []
        dobj2 = []
        dobj1 = []
        dobj_diff = []
        
        
        
            
        

[['alarm system', 'automatic power window']]
alarm system EXCLUDES automatic power window
[['alarm system', 'exterior monitoring']]
alarm system EXCLUDES exterior monitoring
[['interior mirror', 'exterior monitoring']]
interior mirror EXCLUDES exterior monitoring
[['central locking system', 'automatic power window']]
central locking system EXCLUDES automatic power window
[['central locking system', 'manual power window']]
central locking system EXCLUDES manual power window
[['automatic power window', 'LED alarm system']]
automatic power window EXCLUDES LED alarm system
[['automatic power window', 'central alarm system']]
automatic power window EXCLUDES central alarm system
[['exterior monitoring', 'LED alarm system']]
exterior monitoring EXCLUDES LED alarm system
[['exterior monitoring', 'central alarm system']]
exterior monitoring EXCLUDES central alarm system


In [9]:
print(further_processing)

[]


In [10]:
print(list_process)

[]


In [11]:
dobj1

[]

In [17]:
dobj2

[]

In [24]:
# Method 2
# where we make use of dependencies such as agentive,action and objective to achieve our tasks
a = []
b = []
d = []
e = []
diff = []

for i in list(further_processing):
    further_processing.remove(i)
    list_process.append(i)
    print(list_process)
    for z in list_process:
        for s in sentences:
            if(z[0]) in s:
                print(s)
                for triple in dep.triples():
                    if(triple[1] == 'nsubj'):
                        print('The Agentive is:',triple[2][0])
                        a1 = ' '.join(triple[2][0])
                        print('The Action is:',triple[0][0])
                        a2 = ' '.join(triple[0][0])
                        a.append([a1,a2])
                    if(triple[1]=='advcl'):
                        print('The Goal is:',triple[2][0])
                        a4 = triple[2][0]
                        a3 = ' '.join(triple[2][0])
                    if triple[1] =='dobj' and triple[0][0] == a4:
                        print ('The goal of the sentence is:',a4,triple[2][0])
                        a5 = a4 + " " + triple[2][0]
                        a.append([a5])
                        print(a5)
                    
            if(z[1]) in s:
                print(s)
                for triple in dep.triples():
                    if(triple[1] == 'nsubj'):
                        print('The Agentive is:',triple[2][0])
                        d1 = ' '.join(triple[2][0])
                        print('The Action is:',triple[0][0])
                        d2 = ' '.join(triple[0][0])
                        a.append([a1,a2])
                    if(triple[1]=='advcl'):
                        print('The Goal is:',triple[2][0])
                        d4 = triple[2][0]
                        d3 = ' '.join(triple[2][0])
                    if triple[1] =='dobj' and triple[0][0] == a4:
                        print ('The goal of the sentence is:',a4,triple[2][0])
                        d5 = a4 + " " + triple[2][0]
                        d.append([d5])
                        print(d5)
                        
        for i in a:
            for z in i:
                b.append(z)
        
        for j in d:
            for g in j:
                e.append(g)
            
        for element in b:
            if element in e:
                diff.append(element)
                print(len(diff))
            
        if len(dobj_diff) == 0:
            print(z[0],'EXCLUDES',z[1])
        else:
            print(z[0],'REQUIRE',z[1])
                       
                        
                        
        list_process = []
        b = []
        d = []
        diff = []
    
    
    
    
                    
    
